In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/Hackathon - Sound of AI/Code/vqvae'

/content/drive/MyDrive/Hackathon - Sound of AI/Code/vqvae


In [3]:
%%capture
!pip install natsort

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import argparse
import utils
from models.vqvae import VQVAE
import os
import librosa
from natsort import natsorted
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio

parser = argparse.ArgumentParser()

"""
Hyperparameters
"""
timestamp = utils.readable_timestamp()

parser.add_argument("-f")

# DATASET
# Reminder to load audios to local disk , and thus path will be local
parser.add_argument("--audio_folder_path",  type=str, default='/content/drive/MyDrive/Hackathon - Sound of AI/Code/Dataset/Augmented-ESC/audios/')
parser.add_argument("--sample_rate", type=int, default=22050)
parser.add_argument("--train_val_split", type=float, default=0.8)
parser.add_argument("--lazy", type=bool, default=True)
parser.add_argument("--n_fft", type=int, default=2048)
parser.add_argument("--win_len", type=int, default=2000)
parser.add_argument("--hop_len", type=int, default=500)

# DATALOADER
parser.add_argument("--batch_size", type=int, default=32)
parser.add_argument("--num_workers", type=int, default=1)

# MODEL
parser.add_argument("--n_hiddens", type=int, default=128)
parser.add_argument("--n_residual_layers", type=int, default=2)
parser.add_argument("--n_residual_hiddens", type=int, default=32)
parser.add_argument("--embedding_dim", type=int, default=64)
parser.add_argument("--beta", type=float, default=.25)
parser.add_argument("--n_embeddings", type=int, default=2048)

# TRAINING
# whether or not to save model

parser.add_argument("-save", action="store_true", default=True)
parser.add_argument("--filename",  type=str, default=timestamp)
parser.add_argument("--learning_rate", type=float, default=3e-4)
parser.add_argument("--log_interval", type=int, default=250)
parser.add_argument("--n_updates", type=int, default=500000)


args = parser.parse_args()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

training_data, validation_data, training_loader, validation_loader = utils.load_data_and_data_loaders(
    args.audio_folder_path, args.sample_rate, args.train_val_split, args.lazy, args.n_fft, args.win_len,
    args.hop_len, args.num_workers, args.batch_size)

model = VQVAE(args.n_hiddens, args.n_residual_hiddens,
              args.n_residual_layers, args.n_embeddings, args.embedding_dim, args.beta).to(device)


In [36]:
PATH = '/content/drive/MyDrive/Hackathon - Sound of AI/Code/vqvae/results/vqvae_data_sun_jul_10_01_10_20_20228000.pth'
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu'))['model'])
model.eval()
batch = next(iter(validation_loader)).unsqueeze(1)
batch = batch.to(device)
_,output,_ = model(batch)

In [37]:
idx = 6

In [38]:
out_model = output[idx].squeeze(0).cpu().detach().numpy()
out_real = batch[idx].squeeze(0).cpu().detach().numpy()
out_test = np.random.rand(1025,442)

In [39]:
Audio(training_data._postprocess(out_real), rate = 22050)

In [40]:
Audio(training_data._postprocess(out_model), rate = 22050)

In [41]:
Audio(training_data._postprocess(out_test), rate = 22050)